# [NAPARI](https://napari.org/)
## attempting to make color-coded movies for calcium imaging data

In [ ]:
## to install
# !pip install napari[all]

In [ ]:
import napari
from magicgui import magicgui
from enum import Enum
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import skimage
from skimage.io import imread
from skimage.measure import regionprops_table
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
from tqdm import tqdm
import time
import sys
from plotParams import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [ ]:
def loadMovie(dirPath,fString):
    fList = !ls $dPath/*$gene*.tif
    movie = np.asarray([imread(f) for f in fList])
    # sciscan saves 12-bit images into 16-bit data files in the midrange, so that there's 2048 levels
    movie[movie==pow(2,16)]=0
    movie = np.subtract(movie,pow(2,15))
    # movie[movie==(pow(2,16)-pow(2,15))]=0
    return movie

In [ ]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

In [ ]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Load movies to try to identify baseline<a id='F0Extract' name='F0Extract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
def loadMovie(dPath,fString):
    fList = !ls $dPath/*$fString*.tif
    movie = np.asarray([imread(f) for f in fList])
    # sciscan saves 12-bit images into 16-bit data files in the midrange, so that there's 2048 levels
    movie[movie==pow(2,16)]=0
    movie = np.subtract(movie,pow(2,15))
    movie[movie>=65000]=0 #HACK!!!
    return movie

def removeLayer(viewer,lName):
    lNames = []
    for l in viewer.layers:
        lNames.append(l.name)
    if lName in lNames:
        viewer.layers.remove(viewer.layers[lName])
    

In [ ]:
dPath = "/Users/angueyraaristjm/Documents/eelImaging/2Pdata/20220615_5dpf_L02_01/"
fString = 'vamp2'
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()
movie = loadMovie(dPath,fString)
viewer.add_image(movie, contrast_limits=[0,2048])
# viewer.add_image(movie)


In [ ]:
tlims = [1,4];
# removeLayer(viewer,'mean')
l = viewer.layers['movie']
mean = l.data[tlims[0]:tlims[1]].mean(axis=0)
viewer.add_image(mean, blending='additive', contrast_limits=[0,2048], colormap = l.colormap, name = "mean")


# removeLayer(viewer,'dF')
dF = np.empty(movie.shape)
for t in range(movie.shape[0]):
    dF[t,:,:] = movie[t,:,:]-mean
    
viewer.add_image(dF, blending='additive', contrast_limits=[-2048,2048], colormap = 'twilight', name = "dF")

In [ ]:
viewer.layers.remove(viewer.layers['dF_binned'])

dF_binned = skimage.measure.block_reduce(dF, block_size=(2, 1, 1), func=np.mean)
viewer.add_image(dF_binned, blending='additive', colormap = 'twilight', name = "dF_binned")

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose as batch analysis
Usually good parameters are :
1. xOPS:GFP: diameter = 20; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry: diameter = 40; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    
flow_threshold = 0.6 will give more ROIs

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

In [ ]:
### collect file paths
wt_gene = 'wt';
wt_fPaths = [
    '20220408_5dpf_xOG_aCT_uninj_L01_002',
    '20220408_5dpf_xOG_aCT_uninj_L01_004',
    '20220408_5dpf_xOG_aCT_uninj_L02_002',
    '20220408_5dpf_xOG_aCT_uninj_L02_004',
    '20220408_5dpf_xOG_aCT_uninj_L03_002',
    '20220408_5dpf_xOG_aCT_uninj_L04_002',
    '20220408_5dpf_xOG_aCT_uninj_L04_004',
    '20220408_5dpf_xOG_aCT_uninj_L05_002',
    '20220408_5dpf_xOG_aCT_uninj_L05_004',
    '20220408_5dpf_xOG_aCT_uninj_L06_002',
    '20220408_5dpf_xOG_aCT_uninj_L06_004',
    '20220408_5dpf_xOG_aCT_uninj_L07_002',
    '20220408_5dpf_xOG_aCT_uninj_L08_002',
    '20220408_5dpf_xOG_aCT_uninj_L09_002',
    '20220408_5dpf_xOG_aCT_uninj_L09_004',
    '20220408_5dpf_xOG_aCT_uninj_L10_002',
    '20220408_5dpf_xOG_aCT_uninj_L11_002',
    '20220408_5dpf_xOG_aCT_uninj_L11_004',
    '20220408_5dpf_xOG_aCT_uninj_L12_002',
]

# ### collect file paths
cr_gene = 'sall1a';
cr_fPaths = [
    '20220408_5dpf_xOG_aCT_sall1aF0_L01_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L01_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L02_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L02_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L03_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L03_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L04_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L04_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L05_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L05_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L06_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L06_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L07_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L07_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L08_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L08_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L09_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L10_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L11_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L11_004',
    '20220408_5dpf_xOG_aCT_sall1aF0_L12_002',
    '20220408_5dpf_xOG_aCT_sall1aF0_L12_004',
]


# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 35,#40,
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

In [ ]:
print('Starting analysis:\n')
with tqdm(total=len(wt_fPaths)*3, file=sys.stdout) as progBar:
    for fPath in wt_fPaths:
        print('\n' + fPath)
        startTime = time.time()
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + photoreceptors[0] + '_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + photoreceptors[1] + '_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'N_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
    
print('\nFinished cellpose batch analysis')

print('Starting analysis:\n')
with tqdm(total=len(cr_fPaths)*3, file=sys.stdout) as progBar:
    for fPath in cr_fPaths:
        print('\n' + fPath)
        startTime = time.time()
        dOut = dAnalysis + cr_gene + '/' + fPath + '/' + photoreceptors[0] + '_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + cr_gene + '/' + fPath + '/' + photoreceptors[1] + '_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
        dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'N_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
    
print('\nFinished cellpose batch analysis')

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# viewer = napari.Viewer()
viewer.layers.select_all(); viewer.layers.remove_selected()

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRsall1aF0s/xOGaCT/"

# open file
dPath = "/Volumes/angueyra04/LiImaging/20220408_sall1aF0_xOG_aCT/"

fPath = "20220408_5dpf_xOG_aCT_uninj_L01_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L01_004"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L02_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L02_004"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L03_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L04_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L04_004"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L05_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L05_004"; gene = 'wt'; # ripped eye but zoom in ok
fPath = "20220408_5dpf_xOG_aCT_uninj_L06_002"; gene = 'wt'; # curved and slightly ventral
fPath = "20220408_5dpf_xOG_aCT_uninj_L06_004"; gene = 'wt'; # curved, had to go deep in Z
fPath = "20220408_5dpf_xOG_aCT_uninj_L07_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L08_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L09_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L09_004"; gene = 'wt'; # dim and somewhat dorsal.
fPath = "20220408_5dpf_xOG_aCT_uninj_L10_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L11_002"; gene = 'wt';
fPath = "20220408_5dpf_xOG_aCT_uninj_L11_004"; gene = 'wt'; # eye ventrally rotated and with small rip but still near ONH
fPath = "20220408_5dpf_xOG_aCT_uninj_L12_002"; gene = 'wt'; # this is dorsal retina. Exclude.

fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L01_002"; gene = 'sall1a'; # this looks like lor junior
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L01_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L02_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L02_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L03_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L03_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L04_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L04_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L05_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L05_004"; gene = 'sall1a'; # slightly dorsal
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L06_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L06_004"; gene = 'sall1a'; # dorsal
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L07_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L07_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L08_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L08_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L09_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L10_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L11_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L11_004"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L12_002"; gene = 'sall1a';
fPath = "20220408_5dpf_xOG_aCT_sall1aF0_L12_004"; gene = 'sall1a';


# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

photoreceptors = ['R','C','N']
colorList = ['green','magenta','bop blue','orange']

# load mips
mipO = 0.65
for pr,cl in zip(photoreceptors,colorList):
    viewer.open(dOut + pr + "_mip.tiff", plugin='builtins', colormap = cl, blending='additive', opacity=mipO);

# load segmentations
for pr in photoreceptors:
    segData = np.load(dOut + pr + '_mip' + '_seg.npy', allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name = pr + '_seg',blending='additive');
    viewer.layers[pr + '_seg'].preserve_labels = True;

# tweak display to help with segmentation
pr = photoreceptors[0]
# viewer.layers[pr + '_mip'].opacity=0.40;viewer.layers[pr + '_mip'].gamma=1.3;viewer.layers[pr + '_mip'].contrast_limits=[150,2000]
viewer.layers.selection.select_only(viewer.layers[pr + '_seg'])

pr = photoreceptors[1]
viewer.layers[pr + '_mip'].visible = True; viewer.layers[pr + '_seg'].visible = False;
pr = photoreceptors[2]
viewer.layers[pr + '_mip'].visible = False; viewer.layers[pr + '_seg'].visible = False;



print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

pr = photoreceptors[0]
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = pr + '_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(photoreceptors[0]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = pr + '_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(pr))
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(photoreceptors[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = pr + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(pr))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = pr + '_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(pr))

---

### resave curated segmentation after napari-ing around (R_mip first)

In [ ]:
# Doing 2 channel segmentation
baseName = photoreceptors[0]
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

viewer.layers[photoreceptors[1] + '_mip'].visible = True
viewer.layers[photoreceptors[1] + '_seg'].visible = True
viewer.layers[photoreceptors[1] + '_seg'].contour = 0
viewer.layers[photoreceptors[0] + '_mip'].visible = False
viewer.layers[photoreceptors[0] + '_seg'].visible = False
# viewer.layers['N_mip'].visible = True

viewer.add_points(size=10, name = 'U_missing', symbol="o", face_color='#B73AB9', edge_color='#FFFFFF')


@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = photoreceptors[1] + '_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(photoreceptors[1]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(photoreceptors[1]))

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[0] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(photoreceptors[0]))

        
@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = photoreceptors[1] + '_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(photoreceptors[1]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = photoreceptors[1] + '_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(photoreceptors[1]))

### Save Cone segmentation (and UV-cone nuclei if possible)

In [ ]:
# baseName = photoreceptors[1]
# lname = baseName + '_seg'
# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
# print('Done with ' + lname + ' for ' + fPath)

# If NUCS mip revealed UV cones
baseName = 'U'
lname = baseName + '_missing'
l = viewer.layers[lname]; l.save(dOut + lname);
print("Image has {0} {1}-cone nuclei".format(len(viewer.layers[lname].data),baseName))

### Reload curated segmentations after saving

In [ ]:
# remove all except mips
viewer.layers.remove(viewer.layers['U_missing'])

n_mips = 3;
if len(viewer.layers)>n_mips:
    for l in viewer.layers[n_mips:]:
        viewer.layers.remove(l)

viewer.open(dOut + photoreceptors[0] + '_seg_curated.tiff', name=photoreceptors[0] + '_seg', plugin='builtins',blending='additive');
viewer.layers[photoreceptors[0] + '_seg'].contour = 5; viewer.layers[photoreceptors[0] + '_mip'].visible = True
# viewer.open(dOut + photoreceptors[1] + '_seg_curated.tiff', name=photoreceptors[1] + '_seg', plugin='builtins',blending='additive');

print(fPath)
nGreen = len(np.unique(viewer.layers[photoreceptors[0] + '_seg'].data))-1
print("Image has {0} {1}ods".format(nGreen,photoreceptors[0]))
# nRed = len(np.unique(viewer.layers[photoreceptors[1] + '_seg'].data))-1
# print("Image has {0} {1} cones".format(nRed,photoreceptors[1]))

### [Go back and run next one &uarr;](#manualCuration)
---

### reload mip and eye marks in case there are doubts

In [ ]:

nMIP = 3 + 1
viewer.layers['R_seg'].visible = False;
viewer.layers['R_mip'].visible = False;
viewer.layers['N_mip'].visible = False;
viewer.layers['C_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeR'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeR'
    viewer.layers[nMIP+0].opacity = 0.66
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeR'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeC'

eyeMarksData  = np.loadtxt(dOut + 'eyeMarks.csv',delimiter=',',skiprows=1,usecols=[1,2])
eyeMarks = viewer.add_points(eyeMarksData,size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')

In [ ]:
# # manual merging of labels
# mergeLabels = [167,168]
# # lname = 'M_seg'
# lname = 'L_seg'
# if viewer.layers[lname].visible == False:
#     print('Layer not active')
# else:
#     tempd = viewer.layers[lname].data
#     tempd[tempd == mergeLabels[1]]=mergeLabels[0]
#     viewer.layers[lname].data = tempd
#     print('Merged in ' + lname)

In [ ]:
# # manually delete label
# selLabel = 49
# lname = 'L_seg'
# tempd = viewer.layers[lname].data
# tempd[tempd == selLabel]=0
# viewer.layers[lname].data = tempd

# for selLabel in np.arange(15,300):
#     lname = 'L_seg'
#     tempd = viewer.layers[lname].data
#     tempd[tempd == selLabel]=0
#     viewer.layers[lname].data = tempd

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
# viewer = napari.Viewer()
viewer.layers.select_all(); viewer.layers.remove_selected()
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
dPath = '/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRlrrfip1aF0s/'
fName = "CRlrrfip1aF0s_combinedCounts.csv"

dPath = '/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRxbp1F0s/'
fName = "CRxbp1F0_counts.csv"

dPath = '/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRsall1aF0s/'
fName = "CRsall1aF0_counts.csv"

df = pd.read_csv(dPath + fName)
df.head(5)

In [ ]:
geneList = ['wt','sall1a']
nGenes = np.size(geneList)
photoreceptors = ['R','U']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','sall1a']
photoreceptors = ['R','U']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/m2Gs2C/"
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_m2Gs2C_Counts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")